# Penguins Synthetic Data Generation and Clustering

# ****

# Load data and transform

In [122]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from scipy.stats import shapiro
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler
from sklearn.cluster import KMeans, SpectralClustering, MeanShift, estimate_bandwidth
from yellowbrick.cluster.elbow import kelbow_visualizer
from yellowbrick.cluster import SilhouetteVisualizer

In [123]:
data = pd.read_csv("../input/palmer-archipelago-antarctica-penguin-data/penguins_lter.csv")

In [124]:
data = data.drop(["studyName", "Sample Number", "Individual ID", "Comments", "Stage", "Region"], axis=1)

In [125]:
data.head()

In [126]:
data['Date Egg'] =  pd.to_datetime(data['Date Egg'], format='%m/%d/%y')
data['Date Egg'] = pd.DatetimeIndex(data['Date Egg']).month
data = data.rename(columns={'Date Egg': 'MonthEgg', "Clutch Completion": "ClutchCompletion", "Culmen Length (mm)": "CulmenLength", "Culmen Depth (mm)": "CulmenDepth", "Flipper Length (mm)": "FlipperLength", "Body Mass (g)": "BodyMass", "Delta 15 N (o/oo)": "Delta15N", "Delta 13 C (o/oo)": "Delta13C"})

In [127]:
data.info()

In [128]:
# Female = 1 and Male = 0
data = data[data.Sex != "."]
data.Sex.replace(to_replace=dict(FEMALE=1, MALE=0), inplace=True)
data = data.dropna()
data.info()

In [129]:
encoder = OrdinalEncoder()
data[["Island", "ClutchCompletion"]] = encoder.fit_transform(data[["Island", "ClutchCompletion"]])
data.info()

In [130]:
data.head()

In [131]:
scaler = MinMaxScaler()
columns = ["Island", "ClutchCompletion", "MonthEgg", "CulmenLength", "CulmenDepth", "FlipperLength", "BodyMass", "Delta15N", "Delta13C"]
data[columns] = scaler.fit_transform(data[columns])

# Feature Engeneering

In [132]:
data.describe()

In [133]:
data.info()

In [134]:
sns.pairplot(data)

# Visualizations

In [136]:
#Visualize correlations for optimizing visuals
correlations = data.corr(method="pearson")
plt.figure(figsize=(10, 8))
sns.heatmap(correlations, vmin= -1, cmap="coolwarm", annot=True)

In [137]:
plt.subplot(1, 2, 1)
fig1 = sns.countplot(data=data, x="Sex", palette='flare')
fig1.set_xticklabels(["male", "female"])
plt.show()

In [138]:
plt.figure(figsize=(18, 12))
plt.subplot(2, 3, 1)
sns.scatterplot(x=data["CulmenDepth"], y=data["CulmenLength"], hue=data["Sex"], palette='flare')
plt.subplot(2, 3, 2)
sns.kdeplot(x=data.Delta13C, y=data.Delta15N, hue=data["Island"], shade=False, bw_adjust=.5, palette='flare')
plt.subplot(2, 3, 3)
sns.kdeplot(x=data["CulmenDepth"], y=data["CulmenLength"], hue=data["Island"], shade=False, bw_adjust=.5, palette='flare')
plt.subplot(2, 3, 4)
sns.scatterplot(x=data["FlipperLength"], y=data["BodyMass"], hue=data["Sex"], palette='flare')
plt.subplot(2, 3, 5)
sns.kdeplot(x=data["FlipperLength"], y=data["BodyMass"], cmap="Reds", shade=True, bw_adjust=.5, palette='flare')
plt.subplot(2, 3, 6)
sns.kdeplot(x=data["FlipperLength"], y=data["BodyMass"], hue=data["Island"], shade=False, bw_adjust=.5, palette='flare')

## Why is the Delta15N and Delta13C important
Stable isotope values of carbon (delta13C) and nitrogen (delta15N) in blood, feathers, eggshell, and bone have been used in seabird studies since the 1980s, providing a valuable source of information on diet, foraging patterns, and migratory behavior in these birds. These techniques can also be applied to fossil material when preservation of bone and other tissues is sufficient. Excavations of abandoned Adélie penguin (Pygoscelis adeliae) colonies in Antarctica often provide well preserved remains of bone, feathers, and eggshell dating from hundreds to thousands of years B.P. 




[Source](https://pubmed.ncbi.nlm.nih.gov/17620620/)

In [139]:
plt.figure(figsize=(16, 8))
plt.subplot(1, 3, 1)
sns.kdeplot(x=data.Delta13C, y=data.Delta15N, hue=data["Sex"], shade=False, bw_adjust=.5, palette='flare')
plt.subplot(1, 3, 2)
sns.scatterplot(x=data["Delta13C"], y=data["Delta15N"], hue=data["Island"], palette='flare')
plt.subplot(1, 3, 3)
sns.kdeplot(x=data.Delta13C, y=data.Delta15N, hue=data["Island"], shade=False, bw_adjust=.5, palette='flare')

In [140]:
plt.title("Delta13C, Delta15N and the BodyMass colored by Sex")
sns.scatterplot(data=data, x="Delta13C", y="Delta15N", size="BodyMass", hue="Sex", legend=False, sizes=(1, 400), alpha=0.8, palette='flare')

In [141]:
from sklearn.preprocessing import LabelEncoder

data = data.apply(LabelEncoder().fit_transform)

# Hypothesis 1: Not one of the columns are normal distributed

This will be evaluated by using the p-value measurement:

${\displaystyle p=2\min\{\Pr(T\geq t\mid H_{0}),\Pr(T\leq t\mid H_{0})\}}$ for a two-sided test. If distribution ${\displaystyle T}$ is symmetric about zero, then ${\displaystyle p=\Pr(|T|\geq |t|\mid H_{0})}{\displaystyle p=\Pr(|T|\geq |t|\mid H_{0})}$

In [142]:
import warnings
warnings.filterwarnings("ignore")

a=1
plt.figure(figsize=(30, 20))
for i in data.columns:
    plt.subplot(4, 3, a)
    sns.distplot(data[i])
    a += 1
plt.show()

In [143]:
# Shapiro-Wilk Test
for col in data:
    stat, p = shapiro(data[col])
    print("----------------------------------------------")
    print(col)
    print('Statistics=%.3f, p=%.3f' % (stat, p))
    alpha = 0.05
    if p > alpha:
        print('Sample looks Gaussian (fail to reject H0)')
    else:
        print('Sample does not look Gaussian (reject H0)')

# Find out the number of clusters (= species)

## For KMeans

In [144]:
kelbow_visualizer(KMeans(random_state=20), data, k=(1,8))

In [145]:
def sillouethe(data, max_clusters):
    cluster=list(range(2, max_clusters))
    for i in cluster:
        model = KMeans(i)
        visualizer = SilhouetteVisualizer(model)
        visualizer.fit(data)
        
        visualizer.poof()

sillouethe(data, 5)

## For Spectral Clustering

In [146]:
kelbow_visualizer(SpectralClustering(random_state=20), data, k=(1, 8))

## Deciding how many clusters we should use for this data.

According to the Elbow method, we get 4 clusters. However, the Sillhouethe method shows the best results with 2-3 clusters. Therefore, we decided to use the median (= 3). There would also be the possibility to test 2, 3 and 4 clusters and make our decision when we see the results. However, this could lead to wrong decisions, as humans are trained to recognize patterns and such patterns may not exist.

# K-Means Clustering

Note: First time doing a clustering. There might be some errors. Any improvements appreciated!

In [147]:
!pip install sdv

In [148]:
import warnings

import pandas as pd
from pandas_profiling import ProfileReport

from sdv.evaluation import evaluate
from sdv.metrics.tabular import CSTest, KSTest, LogisticDetection, SVCDetection

from sdv.tabular import GaussianCopula

warnings.filterwarnings("ignore")

In [149]:
# !pip show sdv

# Load Data

In [150]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

real_data = data.copy()


In [151]:
print(real_data.shape)
real_data.head()



In [152]:
cat_columns = [
    "Species",
    "Region",
    'Island'
    "Stage",
    "Clutch Completion",
    "Sex",
]

In [153]:
profile = ProfileReport(
    real_data, title="Profiling Real Adult Data", html={"style": {"full_width": True}}
)
profile.to_file("real_data_report_real.html")
profile.to_notebook_iframe()


## Single Table Data

In [154]:
model_copula = GaussianCopula()

In [155]:
%%time
model_copula.fit(real_data)
distributions = model_copula.get_distributions()

In [156]:
from sdv.sampling import Condition

condition = Condition({'Species': 0}, num_rows = 300)

chinstrap = model_copula.sample_conditions(conditions=[condition])

condition = Condition({'Species': 1}, num_rows = 300)

adelie = model_copula.sample_conditions(conditions=[condition])

condition = Condition({'Species': 2}, num_rows = 300)

gentoo = model_copula.sample_conditions(conditions=[condition])

In [157]:
chinstrap

In [158]:
adelie

In [159]:
gentoo

In [160]:
balanced_data = pd.concat([chinstrap, adelie, gentoo], axis = 0, join = "inner")

In [161]:
balanced_data
print(balanced_data.groupby('Species')['Species'].apply(list))

In [162]:
profile_synthetic = ProfileReport(
    balanced_data,
    title="Profiling Synthetic Adult Data",
    html={"style": {"full_width": True}},
)
profile_synthetic.to_file("fake_balanced_data_report_copula.html")
profile_synthetic.to_notebook_iframe()

In [163]:
new_data_copula = model_copula.sample(real_data.shape[0])

In [164]:
profile_synthetic = ProfileReport(
    new_data_copula,
    title="Profiling Synthetic Adult Data",
    html={"style": {"full_width": True}},
)
profile_synthetic.to_file("fake_data_report_copula.html")
profile_synthetic.to_notebook_iframe()

In [165]:
real_data

In [166]:
balanced_data

In [167]:
new_data_copula

In [168]:
%%time
evaluate(
    balanced_data,
    real_data,
    aggregate=False,
)

In [169]:
metrics = ["CSTest", "KSTest"]

In [170]:
%%time
evaluate(
    balanced_data,
    real_data,
    metrics=metrics,
    aggregate=False,
)

In [171]:
evaluate(balanced_data, real_data, metrics=metrics)

In [172]:
%%time
evaluate(
    new_data_copula,
    real_data,
    aggregate=False,
)

In [173]:
metrics = ["CSTest", "KSTest"]

In [174]:
%%time
evaluate(
    balanced_data,
    real_data,
    metrics=metrics,
    aggregate=False,
)

#### Evaluating the experiments

In [175]:
evaluate(new_data_copula, real_data, metrics=metrics)

TRAINING WITH BALANCED DATA

In [176]:
real_data_kmeans = real_data.copy()

i = 1
kmean_clusters = [["CulmenLength", "CulmenDepth"], ["FlipperLength", "BodyMass"], ["Delta15N", "Delta13C"], ["Delta15N", "BodyMass"], 
            ["Delta13C", "BodyMass"], ["CulmenLength", "FlipperLength"]]

plt.figure(figsize=(16, 10))
for cluster in kmean_clusters:
    X_kmean = real_data_kmeans[cluster]
    new_X = new_data_copula[cluster]
    kmeans = KMeans(n_clusters=2, random_state=20)
    kmeans.fit(X_kmean)
    y_kmeans = kmeans.predict(X_kmean)
    y_new = kmeans.predict(new_X)
    real_data_kmeans[f"Cluster{i}"] = y_kmeans
    new_data_copula[f"Cluster{i}"] = y_new
    
    plt.subplot(2, 3, i)
    plt.subplots_adjust(hspace=0.3, wspace=0.3)
    plt.title(f"{i}.) Clustering by: {cluster}")
    sns.scatterplot(X_kmean.loc[:, cluster[0]], X_kmean.loc[:, cluster[1]], c=y_kmeans, s=50, cmap="flare")
    centers = kmeans.cluster_centers_
    sns.scatterplot(centers[:, 0], centers[:, 1], s=200, color="k", alpha=0.5)
    i+=1
plt.savefig("kmeans.png")
plt.show()

In [177]:
# new_data_kmeans = new_data_copula.copy()

# i = 1
# kmean_clusters = [["CulmenLength", "CulmenDepth"], ["FlipperLength", "BodyMass"], ["Delta15N", "Delta13C"], ["Delta15N", "BodyMass"], 
#             ["Delta13C", "BodyMass"], ["CulmenLength", "FlipperLength"]]

# plt.figure(figsize=(16, 10))
# for cluster in kmean_clusters:
#     X_kmean = new_data_kmeans[cluster]
#     kmeans = KMeans(n_clusters=4, random_state=20)
#     kmeans.fit(X_kmean)
#     y_kmeans = kmeans.predict(X_kmean)
#     new_data_kmeans[f"Cluster{i}"] = y_kmeans
    
#     plt.subplot(2, 3, i)
#     plt.subplots_adjust(hspace=0.3, wspace=0.3)
#     plt.title(f"{i}.) Clustering by: {cluster}")
#     sns.scatterplot(X_kmean.loc[:, cluster[0]], X_kmean.loc[:, cluster[1]], c=y_kmeans, s=50, cmap="flare")
#     centers = kmeans.cluster_centers_
#     sns.scatterplot(centers[:, 0], centers[:, 1], s=200, color="k", alpha=0.5)
#     i+=1
# plt.savefig("kmeans.png")
# plt.show()

In [178]:
# balanced_data_kmeans = balanced_data.copy()

# i = 1
# kmean_clusters = [["CulmenLength", "CulmenDepth"], ["FlipperLength", "BodyMass"], ["Delta15N", "Delta13C"], ["Delta15N", "BodyMass"], 
#             ["Delta13C", "BodyMass"], ["CulmenLength", "FlipperLength"]]

# plt.figure(figsize=(16, 10))
# for cluster in kmean_clusters:
#     X_kmean = balanced_data_kmeans[cluster]
#     kmeans = KMeans(n_clusters=4, random_state=20)
#     kmeans.fit(X_kmean)
#     y_kmeans = kmeans.predict(X_kmean)
#     balanced_data_kmeans[f"Cluster{i}"] = y_kmeans
    
#     plt.subplot(2, 3, i)
#     plt.subplots_adjust(hspace=0.3, wspace=0.3)
#     plt.title(f"{i}.) Clustering by: {cluster}")
#     sns.scatterplot(X_kmean.loc[:, cluster[0]], X_kmean.loc[:, cluster[1]], c=y_kmeans, s=50, cmap="flare")
#     centers = kmeans.cluster_centers_
#     sns.scatterplot(centers[:, 0], centers[:, 1], s=200, color="k", alpha=0.5)
#     i+=1
# plt.savefig("kmeans.png")
# plt.show()

In [179]:
X_test = real_data_kmeans.drop(["Species"], axis = 1)
y_test = real_data_kmeans["Species"]

In [180]:
from sklearn.preprocessing import LabelEncoder
print(balanced_data.groupby('Species')['Species'].apply(list))
balanced_data1 = balanced_data.apply(LabelEncoder().fit_transform)
print(balanced_data.describe())
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
logreg = LogisticRegression(solver='liblinear', random_state=0)

parameters = [{'penalty':['l1','l2']}, 
              {'C':[1, 10, 100, 1000]}]



grid_search = GridSearchCV(estimator = logreg,  
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 5,
                           verbose=0)


X = balanced_data_kmeans.drop(["Species"], axis = 1)
print(X.shape)
y = balanced_data_kmeans["Species"]
print(y.shape)
grid_search.fit(X, y)

# best_clf = clf.fit(X,y)
# best_clf.best_estimator_

In [181]:
# examine the best model

# best score achieved during the GridSearchCV
print('GridSearch CV best score : {:.4f}\n\n'.format(grid_search.best_score_))

# print parameters that give the best results
print('Parameters that give the best results :','\n\n', (grid_search.best_params_))

# print estimator that was chosen by the GridSearch
print('\n\nEstimator that was chosen by the search :','\n\n', (grid_search.best_estimator_))

In [182]:
print('GridSearch CV score on test set: {0:0.4f}'.format(grid_search.score(X_test, y_test)))

In [183]:
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

# generate xgboost classifier
xgb = XGBClassifier(learning_rate =0.001,
                    n_estimators=1000,
                    max_depth=100,
                    min_child_weight=2,
                    gamma=0.2,
                    subsample=0.5,
                    colsample_bytree=0.6,
                    scale_pos_weight=1)
model = xgb.fit(X, y)
# fits = xgb.predict(X)
# predscek = xgb.predict(X_test)
# acc_xgbfits = (fits == y_test).sum().astype(float) / len(fits)*100
# acc_xgbcek = (predscek == y_test).sum().astype(float) / len(predscek)*100
# print("XGBoost's prediction accuracy for training data is: %3.2f" % (acc_xgbfits))
# print("XGBoost's prediction accuracy for testing data in sample is: %3.2f" % (acc_xgbcek))
preds = xgb.predict(X_test)
acc_xgbcek = (preds == y_test).sum().astype(float) / len(preds)*100
print("XGBoost's prediction accuracy for testing data in sample is: %3.2f" % (acc_xgbcek))

TRAINING WITH FAKE DATA WITH SIMILAR DISTRIBUTION TO REAL DATA

In [184]:
from sklearn.preprocessing import LabelEncoder
print(new_data_copula.groupby('Species')['Species'].apply(list))
new_data_copula1 = new_data_copula.apply(LabelEncoder().fit_transform)
print(new_data_copula.describe())
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
logreg = LogisticRegression(solver='liblinear', random_state=0)

parameters = [{'penalty':['l1','l2']}, 
              {'C':[1, 10, 100, 1000]}]



grid_search = GridSearchCV(estimator = logreg,  
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 5,
                           verbose=0)


X = new_data_copula1.drop(["Species"], axis = 1)
print(X.shape)
y = new_data_copula1["Species"]
print(y.shape)
grid_search.fit(X, y)

In [185]:
# examine the best model

# best score achieved during the GridSearchCV
print('GridSearch CV best score : {:.4f}\n\n'.format(grid_search.best_score_))

# print parameters that give the best results
print('Parameters that give the best results :','\n\n', (grid_search.best_params_))

# print estimator that was chosen by the GridSearch
print('\n\nEstimator that was chosen by the search :','\n\n', (grid_search.best_estimator_))

print('GridSearch CV score on test set: {0:0.4f}'.format(grid_search.score(X_test, y_test)))

In [186]:
print('GridSearch CV score on test set: {0:0.4f}'.format(grid_search.score(X_test, y_test)))

In [187]:
X_test.shape